### Merge Json

In [ ]:
import io
import os
import json
import random

def json_cat(path):
    files = os.listdir(path=path[:-1])
    
    with open(path + files[0], encoding='utf-8') as f:
        copy = json.load(f)
    with open('testing.json', 'w') as f:
        json.dump(copy, f, ensure_ascii=True)
        
    for file_idx in [185,1,2,3,4,5,400,401,550,551]:
        file = files[file_idx]
        with open(path+file, 'rb') as f:
            data = f.read()
        with open('testing.json', 'rb+') as f:
            f.seek(-1, io.SEEK_END)
            f.write(b', ' + data[1:])

json_cat('C:/Users/CPB06GameN/글을쓰자/PyTorch-master/연습폴더/cleansing_json/')
with open('testing.json', encoding='utf-8') as f:
    data = json.load(f)

### json Generator (ijson)

In [ ]:
import ijson

def json_loader(row_end=100000, block_size=10000, file='testing.json'):
    src_text, tgt_text = [], []
    src_lang, tgt_lang = [], []
    
    with open(file, 'rb') as f:
        parse = ijson.parse(f)
        for idx in range(1, 1+(18*row_end)):
            item, key, cont = next(iter(parse))
            if item == 'item.src_text':
                src_text.append(cont)
            if item == 'item.tgt_text':
                tgt_text.append(cont)
            if item == 'item.src_lang':
                src_lang.append(cont)
            if item == 'item.tgt_lang':
                tgt_lang.append(cont)
                
            if (len(src_text) & len(tgt_text)) == block_size:
                yield src_text, tgt_text, src_lang, tgt_lang
                src_text, tgt_text, src_lang, tgt_lang = [], [], [], []

        if (src_text or tgt_text):
            yield src_text, tgt_text, src_lang, tgt_lang
            
# src_text, tgt_text = json_loader()
data = json_loader(row_end=100000)
print(json_loader)

count = 0
for epoch in range(2):
    for src, tgt, src_lang, tgt_lang in data:
        count += 1
        print(src[0], tgt[0], '\n', count)
    

### Generator Data Train/Test Split

In [ ]:
json_data = json_loader()

def train_test_split(json_data):
    train, validation = [], []
    count = 0
    for src_chunk, tgt_chunk, src_lang, tgt_lang in json_data:
        for src, tgt, slang, tlang in zip(src_chunk, tgt_chunk, src_lang, tgt_lang):
            sent = {'src_text':None, 'tgt_text':None, 'src_lang':None, 'tgt_lang':None}
            if random.uniform(0, 1) <= 0.75:
                sent['src_text'] = src
                sent['tgt_text'] = tgt
                sent['src_lang'] = slang
                sent['tgt_lang'] = tlang
                train.append(sent)
            else:
                sent['src_text'] = src
                sent['tgt_text'] = tgt
                sent['src_lang'] = slang
                sent['tgt_lang'] = tlang
                validation.append(sent)
        count += 1
        yield {'train':train, 'validation':validation}
        train, validation = [], []

### Dynamic Batch Padding with Collate_fn

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def collate_fn_padd(batch):
#     lengths = torch.tensor([t.shape[0] for dic in batch for t in dic.values()]).to(device)
    batch = [t for dic in batch for t in dic.values()]
    ids = torch.nn.utils.rnn.pad_sequence(batch[0::3])
    mask = torch.nn.utils.rnn.pad_sequence(batch[1::3])
    labels = torch.nn.utils.rnn.pad_sequence(batch[2::3])
    ids = ids.contiguous()
    mask = mask.contiguous()
    labels = labels.contiguous()
#     return ids.t(), mask.t(), labels.t()
    return {'input_ids':ids.t(), 'attention_mask':mask.t(), 'labels':labels.t()}

### Map-Style Chunk Generator + Tokenizer 실험 (실패)

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import random

checkpoint = 'facebook/m2m100_418M'
tokenizer = M2M100Tokenizer.from_pretrained(checkpoint, batched=True, truncation=True)

class CustomDataset(Dataset):
    def __init__(self, json_loader, tokenizer, key='train'):
        super().__init__()
        # loop 한 번 돌때마다 10,000개 chunk가 나오는 generator func
        self.data = json_loader()
        self.tokenizer = tokenizer
        self.dataset = {'train':[], 'validation':[]}
        self.key = key

    def __len__(self):
        return 100000
    
    def _encoding(self, chunk_idx):
        count = 0
        for src_chunk, tgt_chunk, src_lang, tgt_lang in self.data:
            count += 1
            if count == chunk_idx:
                for src, tgt, src_lang, tgt_lang in zip(src_chunk, tgt_chunk, src_lang, tgt_lang):
                    
                    tokenized_sent = {'input_ids':None, 'attention_mask':None, 'labels':None}
                    # 매번 lang을 바꾸면 시간이 오래 소요되지 않을까?
                    # config 변경은 별로 안 걸릴 것 같은데??
                    self.tokenizer.src_lang = src_lang
                    self.tokenizer.tgt_lang = tgt_lang
                    
                    inputs = self.tokenizer(src, return_tensors='pt')
                    with self.tokenizer.as_target_tokenizer():
                        labels = tokenizer(tgt).input_ids
                    
                    if random.uniform(0, 1) <= 0.75:
                        tokenized_sent['input_ids'] = inputs.input_ids.reshape(-1)
                        tokenized_sent['attention_mask'] = inputs.attention_mask.reshape(-1)
                        tokenized_sent['labels'] = torch.tensor(labels).reshape(-1)
                        self.dataset['train'].append(tokenized_sent)
                    else:
                        tokenized_sent['input_ids'] = inputs.input_ids.reshape(-1)
                        tokenized_sent['attention_mask'] = inputs.attention_mask.reshape(-1)
                        tokenized_sent['labels'] = torch.tensor(labels).reshape(-1)
                        self.dataset['validation'].append(tokenized_sent)
        return self.dataset
    
    def __getitem__(self, index):
        index = index % 10000
        chunk_idx = index // 10000
        return self._encoding(chunk_idx+1)[self.key][index]


test = CustomDataset(json_loader, tokenizer)

test.__getitem__(2)